In [1]:
import csv
import datetime 
import json
import pandas as pd
import pandas_bokeh as pb
import urllib.request as request

url = "https://opendata.arcgis.com/datasets/f527cf1835294eb280a4e5ee3c5c209c_0.geojson?where=state_name%20%3D%20'new%20york'"


with request.urlopen(url) as response:
    source = response.read()
    data = json.loads(source)

with open ("covid.json", 'w') as outfile:
    json.dump(data, outfile)

with open('covid.json') as json_data:
    d = json.load(json_data)
    print(type(d))
    print(json.dumps(d, indent=3))

<class 'dict'>
{
   "type": "FeatureCollection",
   "name": "State_Representative_Estimates_for_Hospital_Utilization",
   "crs": {
      "type": "name",
      "properties": {
         "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
      }
   },
   "features": [
      {
         "type": "Feature",
         "id": 41,
         "properties": {
            "OBJECTID": 41,
            "state_name": "New York",
            "total_inpatient_beds": 45778,
            "total_icu_beds": 8098,
            "pct_inpatient_bed_utilization": 72.43,
            "pct_inpatients_with_covid": 13.18,
            "pct_icu_bed_utilization": 61.9,
            "icu_beds_used_estimate": 4161,
            "inpatient_beds_used_estimate": 39226,
            "inpatient_beds_used_covid_est": 7137,
            "last_updated": "2020/12/15 14:20:00+00"
         },
         "geometry": {
            "type": "MultiPolygon",
            "coordinates": [
               [
                  [
                     [
                

In [2]:
f = (d['features'])

state = f[0]['properties']['state_name']
in_beds_tot = f[0]['properties']['total_inpatient_beds']
ic_beds_tot = f[0]['properties']['total_icu_beds']
pc_beds_ut = f[0]['properties']['pct_inpatient_bed_utilization']
pc_beds_utcv = f[0]['properties']['pct_inpatients_with_covid']
pc_icu_ut = f[0]['properties']['pct_icu_bed_utilization']
last_upd = f[0]['properties']['last_updated']

print(" Bed Utilization: " + str(pc_beds_ut) + "%", "\n" , "Beds COVID: " + str(pc_beds_utcv) + "%", "\n", "ICU Beds: " + str(pc_icu_ut) + "%", "\n", 
    "Last Updated: " + str(last_upd))

 Bed Utilization: 72.43% 
 Beds COVID: 13.18% 
 ICU Beds: 61.9% 
 Last Updated: 2020/12/15 14:20:00+00


In [16]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure

df = pd.read_csv("base_files/County Stats.csv")

pct_pos = df["Positive"]= (df["New_Positives"] / df["New_Tests"] * 100)
sur_cnt = df[df["County"].isin(["Delaware", "Sullivan", "Ulster", "Orange", "Greene", "Rockland"])]
source = ColumnDataSource(data=sur_cnt)

output_file("bars.html")

names = ['Total_Beds_Utilized', 'COVID', 'ICU']
pcts = [pc_beds_ut, pc_beds_utcv, pc_icu_ut]

z = zip(names,pcts)
d = dict(z)
beds = pd.DataFrame(d, index = [0])

source = ColumnDataSource(data=beds)

tooltips = [("Total", "@pc_beds_ut")]

p_beds = figure(x_range=names, y_range = (0,100), plot_height= 600, plot_width = 800, title="NYS Bed Utilization",
                toolbar_location="right", tooltips = tooltips)

p_beds.vbar(x=names, top=pcts, width=0.5, color=['lightsalmon', 'orange' , 'red'], alpha= 0.8) 

p_beds.xgrid.grid_line_color = None
p_beds.y_range.start = 0

show(p_beds)

In [15]:
z = zip(names,pcts)
d = dict(z)

beds = pd.DataFrame(d, index = [""])

beds

In [12]:
  df = pd.read_csv("base_files/County Stats.csv")

  pct_pos = df["Positive"]= (df["New_Positives"] / df["New_Tests"] * 100)
  sur_cnt = df[df["County"].isin(["Delaware", "Sullivan", "Ulster", "Orange", "Greene", "Rockland"])]

  p_test = beds.plot_bokeh.bar(colormap=['lightsalmon', 'orange' , 'red'], ylim = (0,100), 
                                  title='New_Tests - Sullivan County Area, Data as of: ', xlabel='Bed Utilization', ylabel='% Utilization',
                                  figsize=(800, 200), zooming=False, panning=False, show_figure=False, stacked=False,
                                  hovertool_string="""<h2>Bed Utilization</h2> 
                                                    <h3> Total Beds: @{Total_Beds_Utilized} % </h3>
                                                    <h3> COVID: @{COVID} % </h3>
                                                    <h3> ICU: @{ICU} % </h3>""", alpha=0.6,)

  p_pct = sur_cnt.plot_bokeh.bar(x='County',  y=['Positive'], stacked=False, normed=100, colormap=['red', 'orange'],
                                 title='Percentage Positive Sullivan County Area, Data as of: ', xlabel='County', ylabel='Tests/Cases',
                                 figsize=(800, 600), zooming=False, panning=False, show_figure=False,
                                 hovertool_string="""<h2> @{County} County</h2> 
                                                    <h3> All_Positives: @{All_Positives} </h3>
                                                    <h3> Percentage Positives: @{Positive} % </h3>""", alpha=0.6)

  plot = pb.plot_grid([[p_test, p_pct]], plot_width=800, plot_height=600)

In [6]:
from bokeh.io import output_file, show
from bokeh.plotting import figure

output_file("stacked.html")

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="hover", tooltips="$name @fruits: @$name")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=data,
             legend_label=years)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

In [7]:
filt_key = ['features']

res = [d[key] for key in filt_key]

res

KeyError: 'features'

In [51]:
for key, val in d.items():
    if 'features' in key:
        plot = val

In [50]:
for key, val in d.items():
    print(key)
    if 'features' in key:
        print(key)

type
name
crs
features
features


In [5]:
df = pd.DataFrame.from_dict(res)

df

,0
0,"{'type': 'Feature', 'id': 41, 'properties': {'..."
